In [1]:
!vw -h | head -n10

Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = 
num sources = 1


VW options:
  --random_seed arg                     seed random number generator
  --ring_size arg                       size of example ring

Update options:
  -l [ --learning_rate ] arg            Set learning rate
  --power_t arg                         t power value
  --decay_learning_rate arg             Set Decay factor for learning_rate 


In [2]:
import pandas as pd
import numpy as np

import scipy.sparse
import sklearn.feature_extraction

import matplotlib.pylab as plt
%matplotlib inline

from tqdm import tqdm
import platform

pd.set_option("display.max_rows", 10)
pd.set_option('display.max_columns', 1100)

import os

%pylab inline
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


/home/levabd/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Load and transform data

In [3]:
main_data = np.load('df/main_data.npy').tolist()
values_data = np.load('df/values_data.npy').tolist()
order_data = np.load('df/order_data.npy').tolist()

main_df = pd.DataFrame(main_data)

main_df

,device_class_code,ip,os_code,os_family_code,timestamp,ua_class_code,ua_family_code,ua_string,ua_version
0,desktop,95.181.252.91,windows_7,windows,1485900038,browser,chrome,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,chrome55.0.2883.87
1,smartphone,31.148.3.19,android_4_4,android,1485900079,mobile_browser,chrome_mobile,Mozilla/5.0 (Linux; Android 4.4.2; Zera S Buil...,chrome_mobile49.0.2623.105
2,desktop,188.162.183.66,windows_7,windows,1485900103,browser,firefox,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:51.0) G...,firefox51.0
3,desktop,37.144.52.103,windows_xp,windows,1485900105,browser,ie,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,ie8.0
4,desktop,91.201.178.51,windows_8_1,windows,1485900108,browser,opera,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,opera40.0.2308.81
...,...,...,...,...,...,...,...,...,...
571140,smartphone,89.237.103.208,android_6,android,1488585286,mobile_browser,chrome_mobile,Mozilla/5.0 (Linux; Android 6.0.1; SM-G925F Bu...,chrome_mobile56.0.2924.87
571141,desktop,141.0.12.169,windows_98,windows,1488585377,browser,opera,Mozilla/4.0 (Windows 98; US) Opera 12.16 [en],opera12.16
571142,desktop,131.253.24.147,windows_7,windows,1488585395,browser,ie,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,ie9.0
571143,desktop,176.195.122.100,windows_7,windows,1488585447,browser,ie,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,ie8.0


In [56]:
important_values_keys_set = {
    'Accept', 
    'Accept-Charset', 
    'Accept-Encoding'
}
important_orders_keys_set = {
    'Upgrade-Insecure-Requests',
    'Accept', 
    'If-Modified-Since',
    'Host', 
    'Connection', 
    'User-Agent', 
    'From', 
    'Accept-Encoding' 
}

orders_vectorizer = sklearn.feature_extraction.DictVectorizer(sparse=True, dtype=float)
values_vectorizer = sklearn.feature_extraction.DictVectorizer(sparse=True, dtype=float)

labels = main_df.ua_string.value_counts().index.tolist()
labels.append('NaN')

from lib.parsers.logParser import LogParser

l_parser = LogParser(log_folder='Logs/')
l_parser.reassign_orders_values(order_data, values_data)
full_sparce_dummy = l_parser.prepare_data(orders_vectorizer, values_vectorizer, important_orders_keys_set, important_values_keys_set, fit_dict=True)

#lb = preprocessing.LabelBinarizer(sparse_output=True)
lb = preprocessing.LabelEncoder()
lb.fit(labels)
y = lb.transform(main_df.ua_string.fillna('NaN'))

 11%|█         | 63738/571145 [00:00<00:00, 637377.57it/s]

Sparse dummy orders shape: 
(571145, 50)


100%|██████████| 571145/571145 [00:00<00:00, 654477.30it/s]


Sparse dummy values shape: 
(571145, 516)


In [57]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(full_sparce_dummy, y, test_size=0.33, random_state=42)

In [58]:
y_train

array([27045,  3534,  6761, ..., 35042, 26779, 27045])

In [59]:
train_filename = 'vw/headers.train.vw'
test_filename = 'vw/headers.test.vw'

#y_tr = y_train.reset_index(drop=True)
ftr = open(train_filename, "w")
for row in tqdm(range(X_train.shape[0])):
    s = str(y_train[row]) + " | "
    for i, value in enumerate(X_train.getrow(row).toarray()[0]):
        s += str(i) + ":" + str(value) + " "
    print(s, file=ftr)
    
ftr.close()

100%|██████████| 382667/382667 [11:58<00:00, 532.34it/s]


In [60]:
#y_ts = y_test.reset_index(drop=True)

fts = open(test_filename, "w")
for row in tqdm(range(X_test.shape[0])):
    s = str(y_test[row]) + " | "
    for i, value in enumerate(X_test.getrow(row).toarray()[0]):
        s += str(i) + ":" + str(value) + " "
    print(s, file=fts)
    
fts.close()

100%|██████████| 188478/188478 [06:26<00:00, 530.49it/s]


In [62]:
!head -n 5 vw/headers.train.vw | cut -c 1-50

27045 | 0:0 1:0 2:0 3:0 4:0 5:0 6:0 7:1 8:0 9:0 10
3534 | 0:1 1:0 2:0 3:0 4:0 5:0 6:0 7:0 8:0 9:0 10:
6761 | 0:1 1:0 2:0 3:0 4:0 5:0 6:0 7:0 8:0 9:0 10:
35430 | 0:0 1:0 2:0 3:0 4:0 5:0 6:0 7:0 8:0 9:0 10
24958 | 0:0 1:0 2:0 3:0 4:0 5:0 6:0 7:0 8:0 9:0 10


## Train VW

#### Which loss function should I use?

If the problem is a binary classification (i.e. labels are -1 and +1) your choices should be Logistic or Hinge loss (although Squared loss may work as well). If you want VW to report the 0-1 loss instead of the logistic/hinge loss, add --binary. Example: spam vs non-spam, odds of click vs no-click.
For binary classification where you need to know the posterior probabilities, use --loss_function logistic --link logistic.

If the problem is a regression problem, meaning the target label you're trying to predict is a real value -- you should be using Squared or Quantile loss. 

Example: revenue, height, weight. If you're trying to minimize the mean error, use squared-loss. See: http://en.wikipedia.org/wiki/Least_squares. 

If OTOH you're trying to predict rank/order and you don't mind the mean error to increase as long as you get the relative order correct, you need to minimize the error vs the median (or any other quantile), in this case, you should use quantile-loss. See: http://en.wikipedia.org/wiki/Quantile_regression

**Важно!!!!**
Было бы классно использовать --loss_function logistic --link logistic. Тогда мы бы получили вероятностные предсказания. Но мы к сожалению прогнозируем кучу классов а не бинарную

Вариант добавить юзерагентов или браузеров в классификацию. Но это почти то же самое что наша вторая модель и улучшние скорости с помощью VW весьма бессмысленно

In [66]:
%%time

!vw -d vw/headers.train.vw -f vw/model.vw --loss_function quantile --passes 100 -c -k 2> vw/train.log

CPU times: user 628 ms, sys: 64 ms, total: 692 ms
Wall time: 37.9 s


## Very fast

In [68]:
!head -n20 vw/train.log

final_regressor = vw/model.vw
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = vw/headers.train.vw.cache
Reading datafile = vw/headers.train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
13522.500000 13522.500000            1            1.0 27045.0000   0.0000        5
7644.694092 1766.888184            2            2.0 3534.0000   0.2236        9
9095.930847 10547.167603            4            4.0 35430.0000   0.4614        1
11497.277802 13898.624756            8            8.0 28063.0000   3.5956        9
12342.315807 13187.353813           16           16.0 34323.0000   4.9855        9
13610.973059 14879.630310           32           32.0 29020.0000   9.8947        9
13390.124441 13169.275823           64           64.0 27958.0000   9.1504        9
12545.818460 11701.512480          128          128

In [70]:
%%time

!vw -d vw/headers.test.vw -i vw/model.vw -t -p vw/output.csv --quiet
!head -n3 vw/output.csv

2284.162598
8478.969727
8690.013672
CPU times: user 100 ms, sys: 32 ms, total: 132 ms
Wall time: 6.42 s


In [78]:
y_hat = pd.read_csv('vw/output.csv', header=None)
print(len(y_test))
y_hat[0]

188478


0         2284.162598
1         8478.969727
2         8690.013672
3         8690.013672
4         4325.060547
             ...     
188473    7166.357422
188474    5633.684082
188475    5602.121582
188476    9216.000000
188477    4148.028320
Name: 0, Length: 188478, dtype: float64

In [85]:
y_hat['Real'] = y_test
y_hat.columns = ['Predicted', 'Real']
y_hat = y_hat.round({'Predicted': 0})
y_hat['Success'] = y_hat.Predicted == y_hat.Real
y_hat

,Predicted,Real,Success
0,2284.0,35220,False
1,8479.0,27987,False
2,8690.0,12675,False
3,8690.0,11535,False
4,4325.0,26779,False
...,...,...,...
188473,7166.0,29532,False
188474,5634.0,34242,False
188475,5602.0,23341,False
188476,9216.0,26242,False


In [90]:
y_hat['Success'].value_counts()

False    188478
Name: Success, dtype: int64

### As we see linear models not very effective for multiclass header predictions

I think VW not very useful in this case